In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U transformers accelerate peft bitsandbytes datasets huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 141.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.5 MB/s eta 0:00:00


In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-84379c0g/unsloth_18a2a64dd2544cfeb73855281324abbd
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-84379c0g/unsloth_18a2a64dd2544cfeb73855281324abbd
  Resolved https://github.com/unslothai/unsloth.git to commit bda9e3d39b425f902d29e80c1f2870be7048d9c3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.9 MB/s eta 0:00:00


In [4]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Llama-2-7b-hf", # Or use "unsloth/llama-2-7b" for faster download
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Set up LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2025.11.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments, TrainerCallback
from datasets import Dataset
import torch

# Define test prompts
test_prompts = [
    "Хиймэл оюун гэж юу вэ?",
    "Монголын нийслэл хот аль вэ?",
    # Add more test prompts as needed
]

class InferenceCallback(TrainerCallback):
    def __init__(self, model, tokenizer, test_prompts, alpaca_prompt, inference_steps=100, log_file="inference_log.txt"):
        self.model = model
        self.tokenizer = tokenizer
        self.test_prompts = test_prompts
        self.alpaca_prompt = alpaca_prompt
        self.inference_steps = inference_steps
        self.log_file = log_file

    def on_step_end(self, args, state, control, **kwargs):
        # Run inference every N steps
        if state.global_step % self.inference_steps == 0:
            print(f"\n{'='*80}")
            print(f"Running inference at step {state.global_step}")
            print(f"{'='*80}\n")

            # Switch to inference mode
            self.model.eval()

            for prompt in self.test_prompts:
                inputs = self.tokenizer([
                    self.alpaca_prompt.format(prompt, "")
                ], return_tensors="pt").to("cuda")

                with torch.no_grad():
                    outputs = self.model.generate(
                        **inputs,
                        max_new_tokens=128,
                        use_cache=True,
                        temperature=0.7,
                        do_sample=True
                    )

                response = self.tokenizer.batch_decode(outputs)[0]
                # Extract only the response part
                response_start = response.find("### Response:") + len("### Response:")
                response_text = response[response_start:].strip()

                print(f"Prompt: {prompt}")
                print(f"Response: {response_text}\n")
                print("-" * 80 + "\n")

            # Switch back to training mode
            self.model.train()

        return control

# Load dataset
dataset = Dataset.from_file("/content/drive/MyDrive/llm/llm/data-00000-of-00001.arrow")

alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + tokenizer.eos_token
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

# Create the inference callback
inference_callback = InferenceCallback(
    model=model,
    tokenizer=tokenizer,
    test_prompts=test_prompts,
    alpaca_prompt=alpaca_prompt,
    inference_steps=100  # Run inference every 100 steps
)

In [10]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 6,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        save_strategy = "steps",           # Save at specific step intervals
        save_steps = 100,                  # Save every 100 steps (adjust as needed)
        save_total_limit = 3,              # Keep only the 3 most recent checkpoints
        load_best_model_at_end = False,    # Don't load best model (no evaluation)
    ),
)

trainer.train(resume_from_checkpoint="/content/drive/MyDrive/llm/outputs/checkpoint-2500")

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,322 | Num Epochs = 6 | Total steps = 3,996
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 39,976,960 of 6,778,392,576 (0.59% trained)


Step,Training Loss
2510,0.813700
2520,0.790000
2530,0.765600
2540,0.743500
2550,0.788300
2560,0.811700
2570,0.808800
2580,0.777100
2590,0.855400
2600,0.803100


train/epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▄▃▁▃▆▁▇▃▄▄█▂▃▃▃▃▃▂█▃▇▃▅▄▂▄▃▆▃█▇▆▆▇█▅▅▅█▇
train/learning_rate,█████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁
train/loss,▇███▆▆▆▇▅▆▆▆▆▆▆█▆▆▇▆▆▇▃▂▂▃▄▃▁▁▄▄▃▂▂▃▁▃▂▂
total_flos,4.786438544494264e+17
train/epoch,6
train/global_step,3996
train/grad_norm,1.58863
train/learning_rate,0.0
train/loss,0.6213


TrainOutput(global_step=3996, training_loss=0.2671258596328644, metrics={'train_runtime': 13392.9468, 'train_samples_per_second': 2.384, 'train_steps_per_second': 0.298, 'total_flos': 4.786438544494264e+17, 'train_loss': 0.2671258596328644, 'epoch': 6.0})

In [27]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Хиймэл оюун гэж юу вэ?",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True,
)
print(tokenizer.batch_decode(outputs)[0])

<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Хиймэл оюун гэж юу вэ?

### Response:
Хиймэл оюун бол компьютерийн шинжлэх ухааны салбар бөгөөд өгөгдлөөс суралцаж, түүнийг ашиглах, урвуулахын тулд машин сургалт, байгалийн хэл боловсруулалт, өгөгдөл шинжилгээ зэрэг арга хэмжээг ашигладаг. Энэ нь хиймэл оюун мод, хиймэл оюун эсийн зэрэглэлийг хурдан, нарийвчлалтай илэрхийлэхэд анхаардаг.</s>
